In [1]:
import requests
import cnfg
import json
import pickle
import time
from datetime import datetime
from collections import defaultdict
from urlparse import urlparse
from pymongo import MongoClient
import pandas as pd

In [2]:
config = cnfg.load(".genius_config")
client_key = config["client_key"]
client_secret = config["client_secret"]
access_token = 'uK247ank6jF7DY_E7TEMIdbuhT2IBVsVeFXh6AluxswFUfrlFNfXNu2pUFPrGv2c'
auth = 'Bearer '+access_token
req_start = 'https://api.genius.com'

In [21]:
client = MongoClient()
db = client.music
s = db.songs
# counter
c = s.find()

In [4]:
def pickleLoad(filename):
    with open(filename, 'rb') as f:
        data = pickle.load(f) 
    return data

def pickleDump(data, filename):
    with open(filename, 'wb') as f:
        pickle.dump(data, f)

In [5]:
def api_call(validurl):
    call = validurl
    r = requests.get(call, headers={'Authorization':auth})
    if r.status_code==200:
        return json.loads(r.content)
    else:
        print r.content
        return r.content

In [6]:
songs = [653082, 2396122, 2330692, 2348651, 2295548, 2425602, 2165110, 2312297, 2394578, \
         2305085, 2422646, 2307164, 709256, 2417517, 2401700, 2413367, 2342860, 2439141, \
         2347598, 2432858, 2396122, 2411389, 2433137, 2330692, 2348651, 2295548, 2372618, \
         2425602, 2165110, 2312297, 2394578, 2305085, 2422646, 2307164, 1783676, 709256, \
         2417517, 2401700, 2426609]

In [7]:
artists = [1, 2, 3, 4, 2057, 329739, 13, 589827, 20503, 589828, 47131, 38, 42, 2092, 45, 46, 61488, 56, 59, 64, 16450, 67, 69, 72, 2121, 83, 85, 86, 88, 89, 616538, 92, 241761, 12388, 544871, 105, 107, 112, 117, 118, 217208, 123, 346237, 571520, 12417, 130, 134, 146, 2197, 211094, 157, 158, 61600, 24741, 169, 170, 264368, 10418, 186, 191, 49344, 198, 13347, 2260, 2268, 223, 224, 8417, 166114, 228, 229, 346342, 28906, 350445, 2286, 4337, 33010, 551156, 24823, 10489, 47356, 80125, 257, 649478, 41226, 267, 14607, 272, 233747, 278, 20185, 51248, 8491, 1075, 308, 2358, 641335, 641336, 641337, 641338, 49470, 643393, 579906, 2444, 14666, 162126, 16720, 338, 35155, 196955, 26975, 29024, 141666, 357, 137577, 645493, 645494, 645495, 35193, 37246, 38976, 55684, 2438, 396, 14266, 401, 407955, 405, 543133, 634949, 2470, 537002, 51628, 25005, 43421, 436, 437, 438, 446, 49606, 455, 458, 459, 141781, 471, 391657, 492, 7250, 496, 557559, 506, 324091, 193020, 23037, 68097, 12802, 2571, 2572, 43538, 535, 2585, 546, 554, 57899, 33327, 1119, 606782, 57927, 586, 647765, 606, 4709, 21100, 610928, 154225, 2675, 92265, 4736, 643, 41605, 12939, 662, 328350, 47777, 8866, 6598, 172716, 691, 144054, 695, 12991, 31426, 19139, 21188, 72396, 166608, 27359, 643842, 643843, 643844, 643845, 782, 2831, 8976, 13074, 23315, 17195, 639789, 820, 17207, 835, 25418, 27468, 27474, 27479, 27486, 48956, 27506, 27511, 557945, 568186, 13185, 904, 33260, 162702, 1177, 199577, 922, 9120, 936, 2992, 641969, 953, 27578, 964, 27589, 7111, 156618, 974, 7122, 27603, 27604, 25561, 195546, 988, 27614, 27616, 27618, 29680, 1010, 27638, 27639, 7161, 1020, 27646, 50175, 27648, 27649, 27653, 179206, 451592, 15369, 1039, 15379, 451607, 1052, 646173, 197662, 451615, 451616, 451617, 27683, 27689, 44080, 1073, 214067, 27701, 1078, 27707, 371775, 574528, 27713, 523, 7236, 7237, 539718, 36862, 648273, 648274, 29780, 500824, 11353, 27738, 474207, 1124, 1129, 644204, 37997, 27765, 285818, 49343, 5248, 54401, 11395, 646277, 646278, 23691, 27794, 222357, 154774, 17560, 7321, 27804, 1181, 144557, 27822, 19635, 289976, 27835, 27836, 1218, 48326, 218313, 7031, 49698, 7393, 1255, 1256, 589044, 5367, 13567, 13568, 27916, 589075, 148761, 29978, 1583, 27938, 1319, 7472, 634166, 38206, 451809, 1355, 2274, 42318, 27984, 17746, 648546, 3431, 1393, 1421, 1422, 64930, 634300, 331198, 3526, 183757, 36326, 1513, 30192, 7672, 58017, 30206, 124415, 36352, 11779, 28169, 210443, 7698, 1557, 71193, 44575, 216609, 34340, 14598, 30246, 151087, 517680, 347701, 1601, 388683, 65109, 36440, 271, 177756, 328634, 1630, 1631, 1632, 104044, 290415, 372361, 22161, 338191, 12229, 28615, 32429, 1713, 388799, 944, 181966, 50896, 30425, 69347, 34533, 9972, 642813, 642814, 642815, 14082, 587534, 587535, 1819, 546590, 139040, 182049, 27612, 67379, 67380, 79676, 28480, 310, 149336, 26460, 65377, 34659, 28518, 5996, 22384, 612212, 26507, 491405, 24472, 10144, 455664, 22437, 57262, 57263, 288690, 1972, 38842, 1985, 99318, 39585, 28651, 1362, 2035, 14325, 626678, 64581]

In [8]:
weights = {'artist_id': 10., 'producers':10., 'writers': 10., 'featured_artists': 8., \
          'sampled_songs': 7., 'description_refs': 5., 'references': 4.}

In [101]:
def get_info(song, weights=weights, artists=artists):
    contributors = defaultdict(list)
    cont_weights = defaultdict(float)
    total = 0
    info = s.find({'id':song}, {'_id':0, 'description_refs.artist':1, 'description_refs.song':1, \
                       'sampled_songs':1, 'producers':1, 'writers':1, \
                       'featured_artists':1, 'artist_id':1, 'id':1}).next()
    for e in info:
        if e == 'artist_id':
            contributors['artist_id'].append(int(info[e]))
        if e == 'description_refs':
            if 'artist' in e:
                for a in e['artist']:
                    contributors['description_refs'].append(int(a['id']))
        if e == 'references' in info:
            if 'artist' in e:
                for a in e['artist']:
                    contributors['references'].append(int(a['id']))
        if e == 'producers':
            for a in info[e]:
                contributors['producers'].append(int(a['id']))
        if e == 'writers':
            for a in info[e]:
                contributors['writers'].append(int(a['id']))
        if e == 'featured_artists':
            for a in info[e]:
                contributors['featured_artists'].append(int(a['id']))
        if e == 'sampled_songs':
            for samp in info[e]:
                for sampsong in samp:
                    contributors['sampled_songs'].append(int(sampsong['primary_artist']['id']))
    for artist_type in contributors:
        total += weights[artist_type]
    for artist_type, influencers in contributors.iteritems():
        frac = weights[artist_type]/len(influencers)
        for influencer in influencers:
            cont_weights[influencer] += frac/total
    influences = {(a):(0 if a not in cont_weights.keys() else cont_weights[a]) for a in artists}
    influences_series = pd.Series(data=influences.values(), index=influences.keys())
    return influences_series

In [102]:
df = pd.DataFrame(columns = uniquesongs)

In [83]:
uniquesongs = list(set(songs))

In [110]:
df.shape

(451, 25)

In [104]:
for song in uniquesongs:
    series = get_info(song)
    df[song] = series

In [108]:
for col in df:
    thesum = df[col].sum()
    if thesum<.99999 or thesum>1.00001:
        print col, thesum

In [111]:
df

,2425602,709256,2372618,653082,2295548,2432858,2401700,2426609,2413367,2305085,...,2396122,2307164,2439141,2312297,2348651,2417517,2433137,2165110,1783676,2411389
1,0,0,0,0,0.000000,0,0,0.000000,0.000000,0,...,0,0,0.000000,0.000000,0.000000,0,0.000000,0.0,0,0
2,0,0,0,0,0.000000,0,0,0.000000,0.000000,0,...,0,0,0.000000,0.000000,0.000000,0,0.000000,0.0,0,0
3,0,0,0,0,0.000000,0,0,0.000000,0.000000,0,...,0,0,0.000000,0.000000,0.000000,0,0.000000,0.0,0,0
4,0,0,0,0,0.000000,0,0,0.000000,0.000000,0,...,0,0,0.000000,0.000000,0.000000,0,0.285714,0.0,0,0
2057,0,0,0,0,0.000000,0,0,0.000000,0.000000,0,...,0,0,0.000000,0.000000,0.000000,0,0.000000,0.0,0,0
329739,0,0,0,0,0.000000,0,0,0.000000,0.000000,0,...,0,0,0.000000,0.000000,0.000000,0,0.000000,0.0,0,0
13,0,0,0,0,0.000000,0,0,0.000000,0.000000,0,...,0,0,0.000000,0.000000,0.000000,0,0.000000,0.0,0,0
589827,0,0,0,0,0.000000,0,0,0.000000,0.000000,0,...,0,0,0.000000,0.000000,0.000000,0,0.000000,0.0,0,0
20503,0,0,0,0,0.000000,0,0,0.000000,0.285714,0,...,0,0,0.000000,0.000000,0.000000,0,0.000000,0.5,0,0
589828,0,0,0,0,0.000000,0,0,0.000000,0.000000,0,...,0,0,0.000000,0.000000,0.000000,0,0.000000,0.0,0,0
